In [1]:
from utils import *

tweets, targets, labels, ids = readTweetsOfficial("trump_autolabelled.txt",
                                                  encoding='utf-8')
tweet_tokens = tokenise_tweets(tweets)
target_tokens = tokenise_tweets(targets)
w2vmodel = word2vec.Word2Vec.load("skip_nostop_single_100features_5minwords_5context_big")
# count, dictionary, reverse_dictionary = build_dataset([token for senttoks in tweet_tokens+target_tokens for token in senttoks])  #flatten tweets for vocab construction
transformed_tweets = [transform_tweet(w2vmodel, senttoks) for senttoks in tweet_tokens]
transformed_targets = [transform_tweet(w2vmodel, senttoks) for senttoks in target_tokens]
transformed_labels = transform_labels(labels)
print('Longest tweet', len(max(transformed_tweets, key=len)))
print('Longest target', len(max(transformed_targets, key=len)))

('Longest tweet', 19)
('Longest target', 19)


In [45]:
transformed_tweets[0].shape

(19,)

In [6]:
[w2vmodel.index2word[word] for word in transformed_tweets[0]]

[u"i'd",
 u'love',
 u'see',
 u'#donaldtrump',
 u'live',
 u'life',
 u'average',
 u'immigrant',
 u'unk',
 u"i'd",
 u'surprised',
 u'survived',
 u'week',
 u'#racist',
 u'#racism',
 u'unk',
 u'unk',
 u'unk',
 u'unk']

In [47]:
[w2vmodel.index2word[word] for word in transformed_targets[0]]

[u'donald',
 u'trump',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk',
 u'unk']

In [9]:
w2vmodel.syn0.shape

(36583, 100)

In [37]:
from bicond_tf11 import *
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
learning_rate = 0.0001
batch_size = 70
sample_size = len(transformed_labels)
num_batches = sample_size // batch_size
    
tf.reset_default_graph()
first_seq = tf.placeholder(tf.int32, shape = [None, 19])
second_seq = tf.placeholder(tf.int32, shape = [None, 19])
first_seq_lens = tf.placeholder(tf.int32, shape = [None])
second_seq_lens = tf.placeholder(tf.int32, shape = [None])
labels = tf.placeholder(tf.float64, shape=[None, 3])
dim = 100
num_symbols = len(w2vmodel.vocab)

seq_pair_encoding = create_bicond_embeddings_reader(
    first_seq,
    first_seq_lens,
    second_seq,
    second_seq_lens,
    dim,
    num_symbols
)

W = tf.get_variable(
    'W',
    shape=[200, 3],
    dtype=tf.float64,
    initializer=tf.random_normal_initializer()
)

b = tf.get_variable(
    'b',
    shape=[3],
    dtype=tf.float64,
    initializer=tf.constant_initializer(0.0)
)

logits = tf.tanh(tf.matmul(seq_pair_encoding, W) + b)
losses = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
loss = tf.reduce_sum(losses)
optimizer = tf.train.AdamOptimizer(learning_rate)

optimizer_op = optimizer.minimize(loss)




In [148]:
import time
targets_np = np.array(transformed_targets, dtype=np.int32)
tweets_np = np.array(transformed_tweets, dtype=np.int32)
labels_np = np.array(transformed_labels, dtype=np.float64)
epochs = 8

init = tf.global_variables_initializer()
with tf.Session() as sess:
    total_t = 0
    sess.run(init)
    for epoch in xrange(epochs):
        print('=' * 5 + "[Epoch " + str(epoch) + ']' + '=' * 5)
        for batch in xrange(num_batches):
            start_t = time.time()
            idx = np.random.choice(sample_size, batch_size)
            targets_batch = targets_np[idx]
            targets_lens = 19 * np.ones(batch_size)
            tweets_batch = tweets_np[idx]
            tweets_lens = 19 * np.ones(batch_size)
            labels_batch = labels_np[idx]

            _, loss_np = sess.run(
                [optimizer_op,
                loss],
                feed_dict = {first_seq: targets_batch,
                             first_seq_lens: targets_lens,
                             second_seq: tweets_batch,
                             second_seq_lens: tweets_lens,
                             labels: labels_batch
                            }
            )
            total_t += time.time() - start_t
            if batch % 50 is 0:
                print("Batch number: " + str(batch))
                print("Loss: " + str(loss_np))
                print(str(int(50 * batch_size / total_t)) + " examples/sec")
                total_t = 0


=====[Epoch 0]=====
Batch number: 0
Loss: 78.0637767068
10980 examples/sec
Batch number: 50
Loss: 74.8746969654
409 examples/sec


KeyboardInterrupt: 

In [137]:
batch_size * num_batches

18760

In [138]:
len(tweets_np)

18760